<a href="https://colab.research.google.com/github/Ning9319/SpuCo_Implementation/blob/main/SpuCo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install spuco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=3dc894625e348eeda623befd7f167d18a55219dc725c9d8aa2094398b1983fb9
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [2]:
import torch
import spuco
import spuco.datasets as dsets
from spuco.datasets.base_spuco_dataset import SpuriousFeatureDifficulty
from spuco.utils import set_seed

set_seed(0)

In [3]:

train_set = dsets.spuco_mnist.SpuCoMNIST(root = './data',
                                       classes=[[0,1],[2,3],[4,5], [6,7],[8,9]],
                                       spurious_feature_difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE,
                                       spurious_correlation_strength=0.995)
train_set.initialize()

100%|██████████| 9912422/9912422 [00:00<00:00, 231875334.60it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 32449958.16it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 133984760.77it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 15538767.35it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 48004/48004 [00:10<00:00, 4485.76it/s]


In [4]:
len(train_set)

48004

In [23]:
torch.cuda.is_available()

False

In [5]:
device = torch.device("cpu")

### Step1: Train a model using ERM

In [6]:
from spuco.models import model_factory
model = model_factory('lenet',
                      input_shape = train_set[0][0].shape,
                      num_classes = train_set.num_classes)

In [7]:
from spuco.robust_train import ERM

learning_rate = 1e-2
momentum = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=momentum, nesterov=True)

erm = ERM(model,
          num_epochs=1,
          trainset=train_set,
          batch_size=64,
          optimizer= optimizer,
          verbose = True
          )

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
erm.train()

Epoch 0: 100%|██████████| 751/751 [00:17<00:00, 43.56batch/s, accuracy=100.0%, loss=0.0111]


In [9]:
test_data = dsets.spuco_mnist.SpuCoMNIST(root = './data',
                                       classes=[[0,1],[2,3],[4,5], [6,7],[8,9]],
                                       spurious_feature_difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE,
                                       split = 'test')
test_data.initialize()

100%|██████████| 10000/10000 [00:02<00:00, 4544.96it/s]


In [10]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset = test_data,
    group_partition = test_data.group_partition,
    group_weights = train_set.group_weights,
    batch_size = 64,
    model = model,
    verbose = True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:07,  3.40it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:06,  3.40it/s]

Group (0, 1) Accuracy: 0.0


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:00<00:06,  3.37it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:07<00:00,  3.22it/s]

Group (4, 3) Accuracy: 0.0


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:07<00:00,  3.34it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 100.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 100.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 100.0}

### Step2: Cluster inputs based on the output they produce for ERM

In [11]:
logits = erm.trainer.get_trainset_outputs()

Getting Trainset Outputs: 100%|██████████| 751/751 [00:08<00:00, 90.59batch/s] 


In [12]:
from spuco.group_inference import Cluster, ClusterAlg

cluser = Cluster(
    Z = logits,
    class_labels= train_set.labels,
    cluster_alg=ClusterAlg.KMEANS,
    num_clusters=2,
    verbose=True)

group_partition = cluser.infer_groups()

Clustering class-wise: 100%|██████████| 5/5 [00:00<00:00, 16.71it/s]


In [13]:
for key in sorted(group_partition.keys()):
    print(key, len(group_partition[key]))

(0, 0) 10082
(0, 1) 51
(1, 0) 9623
(1, 1) 49
(2, 0) 8965
(2, 1) 46
(3, 0) 9698
(3, 1) 49
(4, 0) 9393
(4, 1) 48


### Step3: Retrain using "Group-Balancing" to ensure in each batch each group appears equally.

In [14]:
new_model = model_factory('lenet',
                      input_shape = train_set[0][0].shape,
                      num_classes = train_set.num_classes)

In [15]:
new_optimizer = torch.optim.SGD(new_model.parameters(), lr = learning_rate, momentum=momentum, nesterov=True)

In [16]:
from spuco.robust_train import GroupBalanceBatchERM

group_balanced_erm = GroupBalanceBatchERM(
    model=new_model,
    num_epochs=5,
    trainset=train_set,
    group_partition=group_partition,
    batch_size=64,
    optimizer=new_optimizer,
    verbose=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [17]:
group_balanced_erm.train()

Epoch 4: 100%|██████████| 751/751 [00:18<00:00, 41.43batch/s, accuracy=100.0%, loss=4.19e-5]


In [18]:
new_evaluator = Evaluator(
    testset = test_data,
    group_partition = test_data.group_partition,
    group_weights = train_set.group_weights,
    batch_size = 64,
    model = new_model,
    verbose = True
)

new_evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:07,  3.10it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:07,  2.93it/s]

Group (0, 1) Accuracy: 85.81560283687944


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:07,  2.99it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:07<00:00,  3.06it/s]

Group (4, 3) Accuracy: 76.26262626262626


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:08<00:00,  3.11it/s]

Group (4, 4) Accuracy: 99.74747474747475


{(0, 0): 100.0,
 (0, 1): 85.81560283687944,
 (0, 2): 84.39716312056737,
 (0, 3): 78.72340425531915,
 (0, 4): 84.39716312056737,
 (1, 0): 90.70904645476773,
 (1, 1): 99.51100244498778,
 (1, 2): 82.84313725490196,
 (1, 3): 64.46078431372548,
 (1, 4): 57.1078431372549,
 (2, 0): 71.2,
 (2, 1): 76.8,
 (2, 2): 100.0,
 (2, 3): 69.33333333333333,
 (2, 4): 35.026737967914436,
 (3, 0): 67.33668341708542,
 (3, 1): 56.92695214105793,
 (3, 2): 77.58186397984886,
 (3, 3): 99.74811083123426,
 (3, 4): 75.56675062972292,
 (4, 0): 86.64987405541562,
 (4, 1): 63.97984886649874,
 (4, 2): 47.858942065491185,
 (4, 3): 76.26262626262626,
 (4, 4): 99.74747474747475}

In [19]:
new_evaluator.worst_group_accuracy

((2, 4), 35.026737967914436)

In [20]:
new_evaluator.average_accuracy

99.66139282632827

In [21]:
new_evaluator.evaluate_spurious_attribute_prediction()

38.94